In [1]:
from api.chains.off_topic_chain import create_off_topic_chain
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name = "gpt-3.5-turbo", streaming = True)

/Users/danliu/miniconda3/envs/chatbot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
from api.chains.llm_finetune_qa_chain import create_llm_finetun_chain
llm_fine_tune_chain = create_llm_finetun_chain(llm)

In [4]:
llm_fine_tune_chain.invoke({"question": "what is finetuning"})

/Users/danliu/miniconda3/envs/chatbot/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Fine-tuning is the process of updating a pre-trained language model using labeled data to do a specific task, in order to refine its capabilities and improve performance in that particular task or domain. It involves training the model on smaller, specific datasets to make it more specialized and effective for the target task.'

In [21]:
chain = create_off_topic_chain(llm)
chain.invoke({"question": "what is the color of sky"})

'Hi, that seems off-topic.'

In [ ]:
path = "data/llm_finetune/urls/urls.txt"
from service.llama_index_retrive import LlamaRetriever
from service.data_collect import WebPagesToDocuments
from llama_index.embeddings.openai import OpenAIEmbedding
from langchain.chat_models import ChatOpenAI
if_clean_texts = False
docs = WebPagesToDocuments(path = path, clean_texts=if_clean_texts).docs
llm = ChatOpenAI(temperature=0, model_name = "gpt-3.5-turbo", streaming = True)
retriever = LlamaRetriever(db_path="test_db",
                           embeddings_model=OpenAIEmbedding(model="text-embedding-3-small"),
                           docs = docs)

In [17]:
from api.chains.clean_texts_from_url_chain import create_clean_texts_from_url_chain
from langchain_core.runnables.base import RunnableSequence
from typing import List
from langchain.chat_models.base import  BaseChatModel
from llama_index.core.schema import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.web import SimpleWebPageReader
from langchain.chat_models import ChatOpenAI


class DocumentRefiner:
    def __init__(self, llm: BaseChatModel,
                 chunk_size:int = 3000, 
                 chunk_overlap:int = 0):
        self.chain = create_clean_texts_from_url_chain(llm)
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def get_sentence_splitter(self):
        return SentenceSplitter(
            chunk_size=self.chunk_size,
            chunk_overlap=self.chunk_overlap,
        )

    def refine_html_files(self, docs: List[Document]) -> List[Document]:
        cleaned_docs = []
        splitter = self.get_sentence_splitter()
        print("www")
        for doc in docs:
            cleaned_doc_segments = ""
            doc_segments = splitter.get_nodes_from_documents([doc])

            for seg in doc_segments:
                print(seg.text)
                try:
                    
                    response = self.chain.invoke({"question": seg.text})
                    cleaned_doc_segments += response + "\n"
                except Exception as e:
                    print(f"Error processing segment: {e}")

            doc.text = cleaned_doc_segments
            cleaned_docs.append(doc)

        return cleaned_docs

In [18]:
from langchain.chat_models import ChatOpenAI
from service.data_collect import WebPagesToDocuments
path = "data/llm_finetune/urls/urls.txt"
docs = WebPagesToDocuments(path = path, clean_texts=False).docs
llm = ChatOpenAI(temperature=0, model_name = "gpt-3.5-turbo", streaming = True)
cleaner = DocumentRefiner(llm)

www
Join our upcoming webinar âDeriving Business Value from LLMs and RAGs.â

[Register
now](https://www.superannotate.com/webinar?utm_source=alert_bar&utm_medium=banner&utm_campaign=website_alert_bar)

[![superannotate logo](https://assets-global.website-
files.com/612770618d97595db63a9470/6127731d30dc5270fa629b99_logoDark.svg)![](https://assets-
global.website-
files.com/612770618d97595db63a9470/612dea788938e85714b0d752_Logo_Footer.svg)](/)

Platform

[î¤FineTuneCreate top-quality training data across all data
types.](/annotation-tool)

[LLM![](https://assets-global.website-
files.com/612770618d97595db63a9470/64b566d7be24c4d018794a47_arrow_nav_icon.svg)](/llms)[Image![](https://assets-
global.website-
files.com/612770618d97595db63a9470/64b566d7be24c4d018794a47_arrow_nav_icon.svg)](/image-
annotation-tool)[Video![](https://assets-global.website-
files.com/612770618d97595db63a9470/64b566d7be24c4d018794a47_arrow_nav_icon.svg)](/video-
annotation)[Text![](https://assets-global.websit

KeyboardInterrupt: 